# Simulate a temperature controller with an EPICS `swait` record

Learn how to create a simulated [temperature
controller](https://bcda-aps.github.io/bluesky_training/instrument/describe_instrument.html#temperature)
with Bluesky and an EPICS
[swait](https://bcda-aps.github.io/apstools/1.6.17/api/synApps/_swait.html)
record.  We'll show how to simulate the controller in EPICS and use that
simulation as a *positioner* in Bluesky.

In this simulation, the `swait` record provides the computations for the
feedback loop that updates the simulated temperature.

## Connect with a `swait` record

We'll connect with the `gp:userCalc18` PV, an instance of an EPICS `swait`
record in our example IOC.  We'll create the ophyd controller object using the
[SwaitRecord](https://bcda-aps.github.io/apstools/latest/api/synApps/_swait.html#apstools.synApps.swait.SwaitRecord)
structure from the [apstools](https://bcda-aps.github.io/apstools/latest/)
package.

In [1]:
from apstools.synApps import SwaitRecord

controller = SwaitRecord("gp:userCalc8", name="simulator")
controller.wait_for_connection()
print(f"{controller.read()=}\n{controller.read_configuration()=}")

controller.read()=OrderedDict([('simulator_calculated_value', {'value': 39.52739757381552, 'timestamp': 1703633273.495868})])
controller.read_configuration()=OrderedDict([('simulator_description', {'value': 'temperature', 'timestamp': 1703633273.495868}), ('simulator_scanning_rate', {'value': 6, 'timestamp': 1703633273.495868}), ('simulator_disable_value', {'value': 0, 'timestamp': 1703633273.495868}), ('simulator_scan_disable_input_link_value', {'value': 1, 'timestamp': 1703633273.495868}), ('simulator_scan_disable_value_input_link', {'value': 'gp:userCalcEnable.VAL CA MS', 'timestamp': 1703633273.495868}), ('simulator_forward_link', {'value': '0', 'timestamp': 1703633273.495868}), ('simulator_device_type', {'value': 0, 'timestamp': 1703633273.495868}), ('simulator_alarm_status', {'value': 0, 'timestamp': 1703633273.495868}), ('simulator_alarm_severity', {'value': 0, 'timestamp': 1703633273.495868}), ('simulator_new_alarm_status', {'value': 0, 'timestamp': 1703633273.495868}), ('simul

## Create a function to setup the controller

Create a function to configure the `swait` record as a simulated temperature
controller.  The "controller" will update the current computed value (the
*readback*) at `period` based on the setpoint.  Note that `period` here is one
of the preset EPICS `.SCAN` field values.  Pick from any of these values (from
the table at [this
reference](https://epics-base.github.io/epics-base/menuScan.html)):

- `"10 second"`
- `"5 second"`
- `"2 second"`
- `"1 second"`
- `".5 second"`
- `".2 second"`
- `".1 second"`

Be certain to use the exact text string as shown.

The `swait` record will compute the step size based on the difference between
the previous value and the setpoint, limited to the maximum step size.  Random `noise`
is applied to each new computation.  The fields of the `swait` record in
this simulation are described in the next table:

field | description
--- | ---
`.VAL` | readback
`.B` | setpoint
`.A` | previous value
`.C` | noise level
`.D` | maximum change
`.CALC` | calculation expression
`.SCAN` | record scan `period`

The calculation will simulate a feedback loop which reduces the
value of `abs(readback - setpoint)`.

In [2]:
def setup_controller(
    swait,
    setpoint=None,
    label="controller",
    noise=2,
    period="1 second",
    max_change=2
):
    swait.reset()  # remove any prior configuration
    swait.description.put(label)
    swait.channels.A.input_pv.put(swait.calculated_value.pvname)
    if setpoint is not None:
        swait.calculated_value.put(setpoint)  # preset
        swait.channels.A.input_value.put(setpoint)  # readback
        swait.channels.B.input_value.put(setpoint)  # setpoint
    swait.channels.C.input_value.put(noise)  # 2 * noise amplitude
    swait.channels.D.input_value.put(max_change)
    swait.scanning_rate.put(period)
    swait.calculation.put("A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)")

## Setup our controller

Setup our controller with a (randomly-selected) setpoint and scan period.

In [3]:
import random

setup_controller(controller, 10 + 30 * random.random(), period="1 second", label="temperature")

Once configured, the control screen for this `swait` record should look like this screen view:

![swait screen](../_static/userCalc8-as-tc.png)

Watch the controller as it starts for a short time.  The readback should already
be very close (within random noise) to the setpoint value.

In [4]:
import time

t0 = time.time()
for i in range(5):
    time.sleep(1)
    print(
        f"{time.time() - t0:.2f}s:"
        f" readback={controller.calculated_value.get():.2f}"
        f" setpoint={controller.channels.B.input_value.get():.2f}"
    )

1.00s: readback=34.67 setpoint=34.42
2.01s: readback=33.53 setpoint=34.42
3.01s: readback=35.08 setpoint=34.42
4.02s: readback=35.37 setpoint=34.42
5.02s: readback=34.78 setpoint=34.42


## temperature as a positioner

A *positioner* is a device that has both a *readback* (the current measured value) and a *setpoint* (the expected, or demanded, value of the device).  These are available as EPICS PVs from our `swait` record.  We can obtain these directly from our ophyd `controller` object:

signal | swait field | ophyd object
--- | --- | ---
readback | `.VAL` | `controller.calculated_value.pvname`
setpoint | `.B` | `controller.channels.B.input_value.pvname`

We'll create the ophyd `temperature` positioner object using the
[PVPositionerSoftDoneWithStop](https://bcda-aps.github.io/apstools/latest/api/_devices.html#apstools.devices.positioner_soft_done.PVPositionerSoftDoneWithStop)
structure from the [apstools](https://bcda-aps.github.io/apstools/latest/)
package.

In [5]:
from apstools.devices import PVPositionerSoftDoneWithStop

temperature = PVPositionerSoftDoneWithStop(
    "",
    name="temperature",
    readback_pv=controller.calculated_value.pvname,
    setpoint_pv=controller.channels.B.input_value.pvname,
    tolerance=1,
)
temperature.wait_for_connection()
print(f"{temperature.position=}")

temperature.position=34.77883451476771


### Change the setpoint

Watch the readback after the setpoint is changed, until the temperature becomes
`inposition` (`inposition` is a property that reports a `True`/`False` value
determined by `abs(readback - setpoint) <= tolerance`).

Here, we lower the temperature setpoint by 10 from the current readback value.
Then, monitor the readback value until `inposition`.

In [6]:
temperature.setpoint.put(temperature.readback.get() - 10)

t0 = time.time()
while not temperature.inposition:
    time.sleep(1)
    print(
        f"{time.time() - t0:.2f}s:"
        f" readback={temperature.readback.get():.2f}"
        f" setpoint={temperature.setpoint.get():.2f}"
    )

1.00s: readback=33.15 setpoint=24.78
2.01s: readback=31.46 setpoint=24.78
3.01s: readback=29.55 setpoint=24.78
4.01s: readback=28.54 setpoint=24.78
5.02s: readback=26.89 setpoint=24.78
6.02s: readback=25.25 setpoint=24.78


### Move the temperature as a positioner

Here, we treat the `temperature` object as a *positioner*.  

<b>Tip</b>:
In ophyd, a positioner object has a `move()` method and a `position` property.  The `position` property is a shortcut for `readback.get()`.

In [7]:
temperature.position

25.254549781419115

Set the `temperature` to `25` and wait for the *move* to complete.  A `MoveStatus` object is returned by the `move()` method.

<b>Tip</b>: Python prints the value of the last object shown.  In this case,
Python prints the value of the `MoveStatus` object.  It shows
that that the move is done, how long it took, whether the move was successful,
and other information.

In [8]:
temperature.move(25)

MoveStatus(done=True, pos=temperature, elapsed=0.0, success=True, settle_time=0.0)

Make a move *relative* to the current (**readback**) position:

In [9]:
temperature.move(temperature.position + 5)

MoveStatus(done=True, pos=temperature, elapsed=1.2, success=True, settle_time=0.0)

Make a move *relative* to the current **setpoint**:

In [10]:
temperature.move(temperature.setpoint.get() - 5)

MoveStatus(done=True, pos=temperature, elapsed=4.0, success=True, settle_time=0.0)

## Use the `temperature` positioner with a bluesky plan

The `temperature` positioner may be used as a detector or a positioner in a bluesky plan.

First, setup the bluesky objects needed for scanning and reporting.  We won't
need plots nor will we need to save any data.  Also create a convenience function to report the current parameters of the positioner.

In [11]:
from bluesky.run_engine import RunEngine
from bluesky import plans as bp
from bluesky import plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback

bec = BestEffortCallback()
RE = RunEngine()
RE.subscribe(bec)
bec.disable_plots()

def print_position(pos):
    print(
        f"inposition={pos.inposition}"
        f"  position={pos.position:.3f}"
        f"  setpoint={pos.setpoint.get():.3f}"
    )

Set the temperature to `25` using a bluesky plan stub (`bps.mv()`).  Here, `bps.mv()` will set the temperature to an *absolute* value.

A plan stub can be used directly with the `RE()` as shown here, or as part of another bluesky plan.

In [12]:
print_position(temperature)
RE(bps.mv(temperature, 25))
print_position(temperature)

inposition=True  position=26.088  setpoint=25.255
inposition=True  position=24.436  setpoint=25.000


`bps.mvr()` will make a *relative* move.  Decrease the temperature by `5`.

<b>Note</b> that `bps.mvr()` has set the new setpoint to exactly 5 below the
previous *readback* value (not from the previous *setpoint* value).

In [13]:
print_position(temperature)
RE(bps.mvr(temperature, -5))
print_position(temperature)

inposition=True  position=24.436  setpoint=25.000
inposition=True  position=18.976  setpoint=19.436


We can change the setpoint value directly.  But notice that the temperature is not inposition immediately.  This is because we asked for bluesky to wait *only* until setpoint changed, which happened almost instantly.

In [14]:
print_position(temperature)
RE(bps.mvr(temperature.setpoint, 5))
print_position(temperature)

inposition=True  position=18.976  setpoint=19.436
inposition=False  position=18.976  setpoint=24.436


We can measure the readback value (over time) by using `temperature` as a detector.  Here we use the `bp.count` plan, making 5 readings at 1 second intervals.  A data table is printed since this is one of the bluesky plans (`bp`) that create a [run](https://blueskyproject.io/bluesky/multi_run_plans.html#definition-of-a-run) which collects data.

<b>Tip</b>  If this cell is executed immediately after the preceding cell, then it will follow the readback as it approaches the new setpoint.

In [15]:
RE(bp.count([temperature], delay=1, num=5))



Transient Scan ID: 1     Time: 2023-12-26 17:28:14
Persistent Unique Scan ID: '578d480d-0a39-4c80-a95a-2473d92df391'
New stream: 'primary'
+-----------+------------+-------------+
|   seq_num |       time | temperature |
+-----------+------------+-------------+
|         1 | 17:28:14.5 |    18.97621 |
|         2 | 17:28:15.5 |    21.02313 |
|         3 | 17:28:16.5 |    23.44134 |
|         4 | 17:28:17.5 |    24.50802 |
|         5 | 17:28:18.5 |    24.20644 |
+-----------+------------+-------------+
generator count ['578d480d'] (scan num: 1)





('578d480d-0a39-4c80-a95a-2473d92df391',)

To demonstrate the use of `temperature` as a positioner in a scan, we'll need another signal to use as a detector.  We'll create a simple ophyd Signal with a value that does not change.

In [16]:
from ophyd import Signal

det = Signal(name="det", value="123.45")

To see the temperature setpoint reported in the table, set its `kind` attribute to `"hinted"`.  Hinted attributes are shown (and plotted) when they are used as detectors.

In [17]:
temperature.setpoint.kind = "hinted"

Scan `det` vs. `temperature` in 5 steps from 20..40.  See how it is the
*setpoint* which is advanced in even steps.  The `bp.scan()` plan adjusts the
setpoint at each step, waits for the move to complete, then triggers and reads
the detectors.

In [18]:
RE(bp.scan([det], temperature, 20, 40, 5))



Transient Scan ID: 2     Time: 2023-12-26 17:28:19
Persistent Unique Scan ID: 'c61c9483-0561-42f0-878b-66d69546920c'
New stream: 'primary'
+-----------+------------+-------------+----------------------+------------+
|   seq_num |       time | temperature | temperature_setpoint |        det |
+-----------+------------+-------------+----------------------+------------+
|         1 | 17:28:22.5 |    19.27427 |             20.00000 |        123 |
|         2 | 17:28:25.5 |    24.57560 |             25.00000 |        123 |
|         3 | 17:28:29.5 |    30.58123 |             30.00000 |        123 |
|         4 | 17:28:32.5 |    35.68536 |             35.00000 |        123 |
|         5 | 17:28:34.5 |    40.03403 |             40.00000 |        123 |
+-----------+------------+-------------+----------------------+------------+
generator scan ['c61c9483'] (scan num: 2)





('c61c9483-0561-42f0-878b-66d69546920c',)

`bp.rel_scan()` chooses its limits *relative* to the current position.  Here we scan from `-17` to `3`, *relative* to the current position.

In [19]:
print_position(temperature)
RE(bp.rel_scan([det], temperature, -17, 3, 5))

inposition=True  position=40.034  setpoint=40.000


Transient Scan ID: 3     Time: 2023-12-26 17:28:34
Persistent Unique Scan ID: 'd8b4113b-4e6c-4378-a838-95c126814098'
New stream: 'primary'
+-----------+------------+-------------+----------------------+------------+
|   seq_num |       time | temperature | temperature_setpoint |        det |
+-----------+------------+-------------+----------------------+------------+
|         1 | 17:28:41.5 |    23.85823 |             23.03403 |        123 |
|         2 | 17:28:43.5 |    27.23313 |             28.03403 |        123 |
|         3 | 17:28:46.5 |    32.07245 |             33.03403 |        123 |
|         4 | 17:28:49.5 |    37.51014 |             38.03403 |        123 |
|         5 | 17:28:51.5 |    42.50518 |             43.03403 |        123 |
+-----------+------------+-------------+----------------------+------------+
generator rel_scan ['d8b4113b'] (scan num: 3)





('d8b4113b-4e6c-4378-a838-95c126814098',)

## SimulatedTemperatureController device

Combine the setup steps into a single ophyd Device to make a simulator.  Show the support code first:

In [20]:
%pycat ../../../bluesky/user/temperature_controller.py

"""
Create a simulated temperature controller device.

Example with ``swait`` record::

    temperature = SimulatedTemperatureControllerSwait(
        "",
        name="temperature",
        swait_pv="gp:userCalc17",
        tolerance=1,
    )
    temperature.wait_for_connection()
    temperature.setup(25, label="temperature controller")

Example with ``transform`` record::

    temperature = SimulatedTemperatureControllerTransform(
        "",
        name="temperature",
        transform_pv="gp:userTran1",
    )
    temperature.wait_for_connection()
    temperature.setup(25, label="temperature controller", noise=0.2, max_change=0.2, tolerance=0.999)

.. note:: This code is used in the documentation.
   Leave it in the `user` directory.

.. autosummary::

    ~SimulatedTemperatureControllerSwait
    ~SimulatedTemperatureControllerTransform
"""

# TODO: Hoist this code to apstools.  For 1.6.18.
# TODO: After that, refactor and use here.

from apstools.devices import PVPositionerSoftDon

Adjust the Python module import path so this class can be used:

In [21]:
import pathlib, sys

path = pathlib.Path("../../..") / "bluesky" / "user"
sys.path.append(str(path))
from temperature_controller import SimulatedTemperatureControllerSwait

Demonstrate the class by setting up a new temperature controller using a
different `swait` record.

In [23]:
t17 = SimulatedTemperatureControllerSwait(
    "",
    name="t17",
    swait_pv="gp:userCalc17",
    tolerance=1,
)
t17.wait_for_connection()
t17.setup(25, label="t17 controller")

Upgrade the simulator to use a transform record instead.

![transform record](../_static/userTran1-as-tc.png)

In [24]:
from temperature_controller import SimulatedTemperatureControllerTransform

temperature = SimulatedTemperatureControllerTransform(
    "",
    name="temperature",
    transform_pv="gp:userTran1",
)
temperature.wait_for_connection()
temperature.setup(25, label="temperature controller", noise=0.2, max_change=0.2, tolerance=0.999)